# Systems genetics 2020 - Final Project  

TODO: (open issuts)
1. X and Y chr converting to 20?

Import public packges

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import f
import matplotlib.pyplot as plt
import os.path
import sys
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

Import custom packges

In [2]:
sys.path.append('modules')

from utils import print_dim
from preprocessing import data_annotations_merge
from regression import eqtl_analysis
from test_analysis import get_associations, cis_trans_annotation

In [3]:
bold_s = '\033[1m'
bold_e = '\033[0m'

Load data files

In [4]:
liver_exp = pd.read_csv('data/GSE17522_series_matrix_liver.txt', sep = "\t")
brain_exp = pd.read_csv('data/GSE36674_series_matrix_hypothalamus.txt', sep = "\t")

liver_annotations = pd.read_csv('data/annotations_liver_GPL6466-9752.txt', sep = "\t")
brain_annotations = pd.read_csv('data/annotation_brain.annot', sep = "\t")

genotypes = pd.read_excel('data/genotypes.xls', headers=None)
phenotypes = pd.read_excel('data/phenotypes.xls')

mgi = pd.read_csv('data/MGI_Coordinates.Build37.rpt.txt', sep = "\t", error_bad_lines=False, warn_bad_lines=False)

### 2. Gene expression data preprocessing

#### A. Merge data file with annotation file to get your input matrix

Brain

In [5]:
brain_annotations = brain_annotations.rename(columns={'Gene symbol' : 'GENE_SYMBOL'})
brain_matrix = data_annotations_merge(brain_exp, brain_annotations)

Liver

In [6]:
liver_matrix = data_annotations_merge(liver_exp, liver_annotations)

# Keep BXD columns and identifier columns only
liver_bxd_cols = list(liver_matrix.filter(regex=("BXD*")).columns)
id_cols = ['ID', 'GENE_SYMBOL']
liver_matrix = liver_matrix[id_cols + liver_bxd_cols]

# Rename BXD columns
liver_matrix = liver_matrix.rename(columns={col : col.split('_')[1] + '_' + col.split('_')[2] \
                                            for col in liver_matrix.columns.drop(id_cols)})

#### B. Keep only BXD columns that exists in the four files: genotypes, phenotypes and each tissue seperately

In [7]:
genotypes_bxd_cols = list(genotypes.filter(regex=("BXD*")).columns)
phenotypes_bxd_cols = list(phenotypes.filter(regex=("BXD*")).columns)

genotypes_baseline_cols = ["Locus", "Chr_Build37", "Build37_position"]
phenotypes_baseline_cols = ["Phenotype", "Authors", "Year", "Pubmed Id"]

# liver common columns
liver_matrix = liver_matrix.rename(columns = {col : col.split('_')[0] for col in liver_matrix.drop(columns = id_cols)}) 
liver_bxd_cols = list(liver_matrix.filter(regex='BXD').columns)
liver_common_bxd = list(set(genotypes_bxd_cols) & set(phenotypes_bxd_cols) &  set(liver_bxd_cols))
liver_common_bxd.sort()
liver_matrix = liver_matrix[['GENE_SYMBOL'] + liver_common_bxd]
liver_genotypes = genotypes[genotypes_baseline_cols + liver_common_bxd]
liver_phenotypes = phenotypes[phenotypes_baseline_cols + liver_common_bxd]

# brain common columns
brain_matrix = brain_matrix.rename(columns = {col : col.split('_')[0] for col in brain_matrix.drop(columns = id_cols)}) 
brain_bxd_cols = list(brain_matrix.filter(regex='BXD').columns)
brain_common_bxd = list(set(genotypes_bxd_cols) & set(phenotypes_bxd_cols) &  set(brain_bxd_cols))
brain_common_bxd.sort()
brain_matrix = brain_matrix[['GENE_SYMBOL'] + brain_common_bxd]
brain_genotypes = genotypes[genotypes_baseline_cols + brain_common_bxd]
brain_phenotypes = phenotypes[phenotypes_baseline_cols + brain_common_bxd]

#### C. Use only representative genomic loci - Drop duplicated rows (of neighboring loci)

In [8]:
# liver genotypes filtering
snps_org = len(liver_genotypes)

bxd_data = liver_genotypes[liver_common_bxd]
duplicates_indx = bxd_data[bxd_data.shift() != bxd_data].dropna(how='all').index
liver_genotypes = liver_genotypes.loc[duplicates_indx]

print("LIVER Genotypes: Drop duplications:\n#SNPs before: %d   -->   #SNPs after: %d" % (snps_org, len(liver_genotypes)))
liver_genotypes.head(5)

# Brain genotypes filtering
snps_org = len(brain_genotypes)

bxd_data = brain_genotypes[brain_common_bxd]
duplicates_indx = bxd_data[bxd_data.shift() != bxd_data].dropna(how='all').index
brain_genotypes = brain_genotypes.loc[duplicates_indx]

print("LIVER Genotypes: Drop duplications:\n#SNPs before: %d   -->   #SNPs after: %d" % (snps_org, len(brain_genotypes)))
brain_genotypes.head(5)

LIVER Genotypes: Drop duplications:
#SNPs before: 3796   -->   #SNPs after: 1403
LIVER Genotypes: Drop duplications:
#SNPs before: 3796   -->   #SNPs after: 1598


,Locus,Chr_Build37,Build37_position,BXD1,BXD100,BXD101,BXD102,BXD103,BXD11,BXD12,...,BXD80,BXD83,BXD84,BXD85,BXD87,BXD89,BXD90,BXD95,BXD97,BXD99
0,rs6269442,1,3482276,B,B,U,U,U,B,D,...,D,H,B,D,B,B,B,D,B,B
2,rs6376963,1,5008090,B,B,U,U,U,B,D,...,D,H,B,D,B,B,B,D,B,B
3,rs3677817,1,5176059,B,B,U,U,U,B,D,...,D,H,B,D,B,B,B,D,B,B
4,rs8236463,1,5579194,B,B,U,U,U,D,D,...,D,H,B,D,B,B,B,D,B,B
6,rs6298633,1,6820242,B,B,U,U,U,D,D,...,D,H,B,D,B,B,B,D,B,B


#### • Remove rows with no gene identifier, <br> • Remove rows with low maximal value.  <br> • Remove rows with low variance.  <br> • Average multiple rows


Liver

In [9]:
max_treshold = 0.7
var_treshold = 0.2

# Remove rows with no gene identifier
liver_matrix = liver_matrix[~liver_matrix['GENE_SYMBOL'].isna()]
liver_bxd_cols = list(liver_matrix.filter(regex=("BXD*")).columns)

for col in liver_bxd_cols:
    liver_matrix[col] = liver_matrix[col].astype('float64')

# Filter by maximal value
liver_matrix['max'] = liver_matrix.drop(columns=['GENE_SYMBOL']).max(axis=1)
liver_matrix = liver_matrix[liver_matrix['max'] >= max_treshold]
liver_matrix = liver_matrix.drop(columns = 'max')
print("Num of rows after removing all rows with maximal values less than %.2f: %d " % (max_treshold, len(liver_matrix)))

# Filter by variance
liver_matrix['var'] = liver_matrix.drop(columns=['GENE_SYMBOL']).var(axis=1)
liver_matrix = liver_matrix[liver_matrix['var'] >= var_treshold]
liver_matrix = liver_matrix.drop(columns = 'var')
print("Num of rows after removing all rows with variance less than %.2f: %d " % (var_treshold, len(liver_matrix)))

# Group multiple rows by mean
liver_matrix = liver_matrix.groupby('GENE_SYMBOL').agg('mean').reset_index()
print("Num of rows after removing duplicated rows: %d ", len(liver_matrix))

Num of rows after removing all rows with maximal values less than 0.70: 7772 
Num of rows after removing all rows with variance less than 0.20: 1308 
Num of rows after removing duplicated rows: %d  1285


Brain

In [10]:
max_treshold = 9
var_treshold = 0.04

# Remove rows with no gene identifier
brain_matrix = brain_matrix[~brain_matrix['GENE_SYMBOL'].isna()]
brain_bxd_cols = list(brain_matrix.filter(regex=("BXD*")).columns)

for col in brain_bxd_cols:
    brain_matrix[col] = brain_matrix[col].astype('float64')

# Filter by maximal value
brain_matrix['max'] = brain_matrix.drop(columns=['GENE_SYMBOL']).max(axis=1)
brain_matrix = brain_matrix[brain_matrix['max'] >= max_treshold]
brain_matrix = brain_matrix.drop(columns = 'max')
print("Num of rows after removing all rows with maximal values less than %.2f: %d " % (max_treshold, len(brain_matrix)))

# Filter by variance
brain_matrix['var'] = brain_matrix.drop(columns=['GENE_SYMBOL']).var(axis=1)
brain_matrix = brain_matrix[brain_matrix['var'] >= var_treshold]
brain_matrix = brain_matrix.drop(columns = 'var')
print("Num of rows after removing all rows with variance less than %.2f: %d " % (var_treshold, len(brain_matrix)))

# Group multiple rows by mean
brain_matrix = brain_matrix.groupby('GENE_SYMBOL').agg('mean').reset_index()
print("Num of rows after removing duplicated rows: %d ", len(brain_matrix))

Num of rows after removing all rows with maximal values less than 9.00: 9893 
Num of rows after removing all rows with variance less than 0.04: 1444 
Num of rows after removing duplicated rows: %d  1247


#### • Average across different individuals of the same strain (Females and males)

In [11]:
liver_matrix = liver_matrix.rename(columns = {col : col.split('_')[0] for col in liver_matrix.drop(columns = ['GENE_SYMBOL'])}) 
liver_matrix = liver_matrix.set_index(['GENE_SYMBOL']) 
liver_matrix = liver_matrix.groupby(by=liver_matrix.columns, axis=1).mean()
liver_matrix = liver_matrix.reset_index()

brain_matrix = brain_matrix.rename(columns = {col : col.split('_')[0] for col in brain_matrix.drop(columns = ['GENE_SYMBOL'])}) 
brain_matrix = brain_matrix.set_index(['GENE_SYMBOL']) 
brain_matrix = brain_matrix.groupby(by=brain_matrix.columns, axis=1).mean()
brain_matrix = brain_matrix.reset_index()

### 3. eQTL analysis

#### Run  Regression

In [12]:
print_dim(len(liver_matrix), len(liver_genotypes), title="Liver: ")
print_dim(len(brain_matrix), len(brain_genotypes), title="Brain: ")

Liver: 
* Expression matrix size: 1285
* Genotype matrix size: 1403
* Expected num of tests: 1,802,855

Brain: 
* Expression matrix size: 1247
* Genotype matrix size: 1598
* Expected num of tests: 1,992,706



In [13]:
if not os.path.exists("brain_reg_results.csv"):
    brain_reg_results = eqtl_analysis(brain_matrix, brain_genotypes, file_prefix="brain_")
else:
    brain_reg_results = pd.read_csv("brain_reg_results.csv")
    brain_reg_results = brain_reg_results.drop(columns='Unnamed: 0')

In [14]:
if not os.path.exists("liver_reg_results.csv"):
    liver_reg_results = eqtl_analysis(liver_matrix, liver_genotypes, file_prefix="liver_")
else:
    liver_reg_results = pd.read_csv("liver_reg_results.csv")
    liver_reg_results = liver_reg_results.drop(columns='Unnamed: 0')

#### Cis/Trans annotation

In [15]:
brain_reg_results = cis_trans_annotation(brain_reg_results.copy(), mgi)
liver_reg_results = cis_trans_annotation(liver_reg_results.copy(), mgi)

100%|██████████████████████████████████████████████████████████████████████████████| 1285/1285 [05:19<00:00,  4.02it/s]


#### Multiple test correction and associations filtering

In [16]:
brain_associations, brain_num_tests = get_associations(brain_reg_results) # by bonfferoni corrections
brain_associations.to_csv("brain_assoc_eqtl.csv")

liver_associations, liver_num_tests = get_associations(liver_reg_results) # by bonfferoni corrections
liver_associations.to_csv("brain_assoc_eqtl.csv")

In [19]:
def print_stats(assoc_eqtl, num_tests, title=""):
    total = len(assoc_eqtl)
    cis = assoc_eqtl[assoc_eqtl['closeness'] == 'cis']
    trans = assoc_eqtl[assoc_eqtl['closeness'] == 'trans']
    print(bold_s + title + bold_e)
    print(bold_s + "Number of tests: " + bold_e, num_tests)
    print("\n" + bold_s + "Number of different significant eQTLs: " + bold_e, len(assoc_eqtl.SNP.unique()))
    print("From which: \n %d - cis-acting \n %d - trans-acting " % (len(cis.SNP.unique()), len(trans.SNP.unique())))
    print("\n" + bold_s + "Number of total significant eQTLs: " + bold_e, len(assoc_eqtl))
    print("From which: \n %d - cis-acting \n %d - trans-acting " % (len(cis), len(trans)))

In [21]:
print_stats(brain_associations, brain_num_tests, "Brain")
print()
print_stats(liver_associations, liver_num_tests, "Liver")

Brain
Number of tests:  1992706

Number of different significant eQTLs:  472
From which: 
 260 - cis-acting 
 231 - trans-acting 

Number of total significant eQTLs:  868
From which: 
 349 - cis-acting 
 337 - trans-acting 

Liver
Number of tests:  1802855

Number of different significant eQTLs:  481
From which: 
 261 - cis-acting 
 278 - trans-acting 

Number of total significant eQTLs:  840
From which: 
 325 - cis-acting 
 468 - trans-acting 
